In [2]:
from transformers import AutoTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax

label_map = { 0: "İntihar Düşüncesi", 1: "Yeme Bozuklukları", 2:"Uyku Bozuklukları", 3:"Cinsel Bozukluklar", 4: "Bağımlılıklar",5 : "Öfke Kontrol Bozuklukları",
              6: "Borderline", 7: "Psikomatik Bozukluklar", 8: "OKB", 9: "Çocuklarda ve Yetişkinlerde Uyum Problemleri", 10: "Depresyon Vb.", 11: "Aile ve İlişki Sorunları",
              12: "Spor Psikolojisi", 13: "Hiperaktivite ve Nerogelişimsel Bzk.", 14: "Travmaya Bağlı Bozukluklar", 15: "Sapkınlıklar"}

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForSequenceClassification.from_pretrained('atakanyilmaz/bert-turkish-psychological-classification')


text = "Uzun süren ilişkiden aldatılma sonucunda ayrıldım. bunun üstüne kendimi boşlukta hissediyorum ve çok üzgünüm. Aylar geçti ama halen kendimi toparlayamadım."


inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)


model.eval()


with torch.no_grad():
	outputs = model(**inputs)


logits = outputs.logits


scores = softmax(logits, dim=1)


num_labels = model.config.num_labels

top_scores, top_indices = torch.topk(scores, 15)


print("######## Tahmin Edilecek Metin ########\n")
print(text,"\n")
print("######## Tahmin Sonuçları ########\n")
for i in range(3):
    label_index = top_indices[0][i].item()
    label_description = label_map.get(label_index, "Bilinmeyen")
    print(f"{label_description}: %{round(top_scores[0][i].item()*100,2)}")

/home/querriqe/miniconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


######## Tahmin Edilecek Metin ########

Uzun süren ilişkiden aldatılma sonucunda ayrıldım. bunun üstüne kendimi boşlukta hissediyorum ve çok üzgünüm. Aylar geçti ama halen kendimi toparlayamadım. 

######## Tahmin Sonuçları ########

Travmaya Bağlı Bozukluklar: %65.79
Aile ve İlişki Sorunları: %17.71
Cinsel Bozukluklar: %2.3


In [4]:
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained('atakanyilmaz/bert-turkish-psychological-classification')
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)

result = nlp("Uzun süren ilişkiden aldatılma sonucunda ayrıldım. bunun üstüne kendimi boşlukta hissediyorum ve çok üzgünüm. Aylar geçti ama halen kendimi toparlayamadım.")

print(result)

[{'label': 'LABEL_14', 'score': 0.6578572988510132}]
